# Exercise 1

Given the complexity of analyzing visual data like protest pageantry, and the desire to avoid overly simplistic models, a combination of supervised and unsupervised machine learning techniques would likely be most relevant.  Specifically, convolutional neural networks (CNNs), a form of deep learning, would be suitable for image feature extraction.  Since the researcher has captions describing the protest's topic, this labeled data could be used in a supervised learning approach to train a CNN to recognize visual cues associated with different protest themes.  However, the researcher might also want to discover *unforeseen* visual similarities across protests, even those with different stated aims.  For this, unsupervised learning methods like clustering algorithms (e.g., k-means) could be applied to the extracted image features to group protests based on visual similarity, potentially revealing interesting patterns.  The captions could then be used *post hoc* to understand the themes of these visually-derived clusters.  There's ambiguity in how precisely the captions will be used – will they be the *sole* basis for supervised learning, or will they supplement other labeled data?  Also, the researcher could use transfer learning, leveraging pre-trained CNNs on large image datasets, to fine-tune the model for this specific task, potentially reducing the need for extensive labeled data.


# Exercise 2

In [49]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [50]:
# Loading the sample data
data = pd.read_csv('formative_data.csv')

In [51]:
# Train/test split

X = data[['x1', 'x2', 'x3', 'x4', 'x5']]
y = data['outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Here I will create a function to test many different model specifications for a linear regression

In [52]:
# Helper function for transformations

def transform_data(X, spec):
    """
    Transforms the DataFrame X according to a spec string or dictionary.

    Parameters
    ----------
    X : pd.DataFrame
        The original feature DataFrame.
    spec : str
        A label describing the transformation.

    Returns
    -------
    X_transformed : pd.DataFrame
        A new DataFrame of transformed features.
    """

    # Make a copy so we don't modify the original
    X_new = X.copy()

    if spec == 'baseline':
        # Return X as is
        return X_new

    elif spec == 'only_x1':
        # Return a DataFrame with only x1
        return X[['x1']].copy()

    elif spec == 'only_x2':
        return X[['x2']].copy()

    elif spec == 'only_x3':
        return X[['x3']].copy()

    elif spec == 'only_x4':
        return X[['x4']].copy()

    elif spec == 'only_x5':
        return X[['x5']].copy()

    elif spec == 'x1_x2_only':
        return X[['x1','x2']].copy()

    elif spec == 'x1_x2_x3_only':
        return X[['x1','x2','x3']].copy()

    elif spec == 'x1_x2_x3_x4_only':
        return X[['x1','x2','x3','x4']].copy()

    elif spec == 'x1_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        return X_new

    elif spec == 'x2_squared':
        X_new['x2_squared'] = X_new['x2'] ** 2
        return X_new

    elif spec == 'x5_squared':
        X_new['x5_squared'] = X_new['x5'] ** 2
        return X_new

    elif spec == 'x1_and_x5_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x5_squared'] = X_new['x5'] ** 2
        return X_new

    elif spec == 'x1_and_x2_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x2_squared'] = X_new['x2'] ** 2
        return X_new

    elif spec == 'x1_and_x2_and_x5_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x2_squared'] = X_new['x2'] ** 2
        X_new['x5_squared'] = X_new['x5'] ** 2
        return X_new

    elif spec == 'x1_and_x2_and_x3_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x2_squared'] = X_new['x2'] ** 2
        X_new['x3_squared'] = X_new['x3'] ** 2
        return X_new

    elif spec == 'x1_and_x2_and_x3_and_x4_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x2_squared'] = X_new['x2'] ** 2
        X_new['x3_squared'] = X_new['x3'] ** 2
        X_new['x4_squared'] = X_new['x4'] ** 2
        return X_new

    elif spec == 'x1_and_x2_and_x3_and_x4_and_x5_squared':
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x2_squared'] = X_new['x2'] ** 2
        X_new['x3_squared'] = X_new['x3'] ** 2
        X_new['x4_squared'] = X_new['x4'] ** 2
        X_new['x5_squared'] = X_new['x5'] ** 2
        return X_new

    elif spec == 'x1_x2_interaction':
        X_new['x1_x2'] = X_new['x1'] * X_new['x2']
        return X_new

    elif spec == 'x4_x5_interaction':
        X_new['x4_x5'] = X_new['x4'] * X_new['x5']
        return X_new

    elif spec == 'x1_squared_interaction':
        # Combine squaring x1 and adding x1*x2
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x1_x2']      = X_new['x1'] * X_new['x2']
        return X_new

    elif spec == 'log_x1':
        # Handle negatives or zero by shifting
        X_new['log_x1'] = np.log(np.abs(X_new['x1']) + 1)
        return X_new

    elif spec == 'log_x2':
        X_new['log_x2'] = np.log(np.abs(X_new['x2']) + 1)
        return X_new

    elif spec == 'log_all':
        # Log transform all features that can be safely logged
        # We do abs(value) + 1 for each
        for col in X_new.columns:
            X_new[f'log_{col}'] = np.log(np.abs(X_new[col]) + 1)
        return X_new

    elif spec == 'polynomial_degree_2':
        # Use PolynomialFeatures from sklearn
        # This will create 1, x1, x2, x3, x4, x5, plus cross terms and squares
        poly = PolynomialFeatures(degree=2, include_bias=False)
        arr = poly.fit_transform(X_new)
        col_names = poly.get_feature_names_out(X_new.columns)
        return pd.DataFrame(arr, columns=col_names)

    elif spec == 'polynomial_degree_3':
        # Similarly for cubic features
        poly = PolynomialFeatures(degree=3, include_bias=False)
        arr = poly.fit_transform(X_new)
        col_names = poly.get_feature_names_out(X_new.columns)
        return pd.DataFrame(arr, columns=col_names)


    # All features + squares + all pairwise interactions
    elif spec == 'all_features_squares_interactions':
        # For every feature, create a squared column
        for col in X_new.columns:
            X_new[f'{col}_squared'] = X_new[col] ** 2
        # For every pair, create an interaction column
        cols = X_new.columns
        for i in range(len(cols)):
            for j in range(i+1, len(cols)):
                X_new[f'{cols[i]}_x_{cols[j]}'] = X_new[cols[i]] * X_new[cols[j]]
        return X_new

    # Only x1 and x5 + squares + interaction
    elif spec == 'x1_and_x5_squares_interaction':
        X_new = X[['x1','x5']].copy()
        X_new['x1_squared'] = X_new['x1'] ** 2
        X_new['x5_squared'] = X_new['x5'] ** 2
        X_new['x1_x5'] = X_new['x1'] * X_new['x5']
        return X_new

    # Polynomial degree 2 ignoring x3
    elif spec == 'poly_deg2_ignore_x3':
        X_subset = X[['x1','x2','x4','x5']].copy()
        poly = PolynomialFeatures(degree=2, include_bias=False)
        arr = poly.fit_transform(X_subset)
        col_names = poly.get_feature_names_out(X_subset.columns)
        return pd.DataFrame(arr, columns=col_names)

    # Logs + squares for x1, x5
    elif spec == 'log_x1_and_x5_squared':
        X_new = X.copy()
        X_new['log_x1'] = np.log(np.abs(X_new['x1']) + 1)
        X_new['x5_squared'] = X_new['x5'] ** 2
        return X_new

    # Cubic for x1 and x5 only
    elif spec == 'cubic_x1_and_x5_only':
        X_subset = X[['x1','x5']].copy()
        poly = PolynomialFeatures(degree=3, include_bias=False)
        arr = poly.fit_transform(X_subset)
        col_names = poly.get_feature_names_out(X_subset.columns)
        return pd.DataFrame(arr, columns=col_names)

    # Polynomial on x5 plus log x1
    elif spec == 'poly_x5_plus_log_x1':
        # Handle infinities by turning them into NaN
        X_clean = X.replace([np.inf, -np.inf], np.nan).copy()

        # Drop any rows where x1 or x5 is NaN
        # (Or you could impute if you prefer)
        X_clean = X_clean.dropna(subset=['x1','x5'])

        # Keep track of the index that remains
        idx = X_clean.index

        # Polynomial expansion for x5
        poly = PolynomialFeatures(degree=2, include_bias=False)
        arr = poly.fit_transform(X_clean[['x5']])
        col_names = poly.get_feature_names_out(['x5'])

        X_poly = pd.DataFrame(arr, columns=col_names, index=idx)

        # Add log_x1
        X_poly['log_x1'] = np.log(np.abs(X_clean['x1']) + 1)

        # (Optionally) add other columns
        X_poly[['x2','x3','x4']] = X_clean[['x2','x3','x4']]

        return X_poly


    # Interactions among x2, x4, x5 only
    elif spec == 'selective_interactions_x2_x4_x5':
        X_new['x2_x4'] = X_new['x2'] * X_new['x4']
        X_new['x2_x5'] = X_new['x2'] * X_new['x5']
        X_new['x4_x5'] = X_new['x4'] * X_new['x5']
        return X_new

    # Binning x4
    elif spec == 'x4_binned':
        # Example bins for x4 -> adjust to your data
        bins = [-np.inf, 0, 2, np.inf]
        labels = ['low','medium','high']
        X_new['x4_binned'] = pd.cut(X_new['x4'], bins=bins, labels=labels)
        X_new = pd.get_dummies(X_new, columns=['x4_binned'], drop_first=True)
        return X_new

    else:
        raise ValueError(f"Unknown specification: {spec}")

### Now I will test each specification and keep the one with the highest score on th test set

In [53]:
# Define many specifications

specs = [
    'baseline',
    'only_x1',
    'only_x2',
    'only_x3',
    'only_x4',
    'only_x5',
    'x1_x2_only',
    'x1_x2_x3_only',
    'x1_x2_x3_x4_only',
    'x1_squared',
    'x2_squared',
    'x5_squared',
    'x1_and_x5_squared',
    'x1_and_x2_squared',
    'x1_and_x2_and_x5_squared',
    'x1_and_x2_and_x3_squared',
    'x1_and_x2_and_x3_and_x4_squared',
    'x1_and_x2_and_x3_and_x4_and_x5_squared',
    'x1_x2_interaction',
    'x4_x5_interaction',
    'x1_squared_interaction',
    'log_x1',
    'log_x2',
    'log_all',
    'polynomial_degree_2',
    'polynomial_degree_3',
    'all_features_squares_interactions',
    'x1_and_x5_squares_interaction',
    'poly_deg2_ignore_x3',
    'log_x1_and_x5_squared',
    'cubic_x1_and_x5_only',
    'poly_x5_plus_log_x1',
    'selective_interactions_x2_x4_x5',
    'x4_binned'
]

# 5) Train and evaluate each spec

results = []

for spec in specs:
    # Transform training and test data
    X_train_t = transform_data(X_train, spec)
    X_test_t  = transform_data(X_test, spec)

    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X_train_t, y_train)

    # Predict on test set
    y_pred = model.predict(X_test_t)

    # Calculate R² and MSE
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Store (spec, r2, mse)
    results.append((spec, r2, mse))

# Sort results by R² descending
results.sort(key=lambda x: x[1], reverse=True)

# Print out specs, R², and MSE
for spec, r2_val, mse_val in results:
    print(f"Spec: {spec:30s} R²: {r2_val:.4f}  MSE: {mse_val:.4f}")

best_spec, best_r2, best_mse = results[0]
print("\nBest specification:", best_spec)
print("Best R²:", best_r2)
print("Best MSE:", best_mse)


Spec: all_features_squares_interactions R²: 0.9712  MSE: 2.4884
Spec: poly_x5_plus_log_x1            R²: 0.8841  MSE: 10.0054
Spec: log_x1_and_x5_squared          R²: 0.8837  MSE: 10.0369
Spec: x1_and_x2_and_x3_and_x4_and_x5_squared R²: 0.7965  MSE: 17.5642
Spec: x1_and_x5_squared              R²: 0.7963  MSE: 17.5849
Spec: x1_and_x2_and_x5_squared       R²: 0.7961  MSE: 17.6004
Spec: x1_and_x2_and_x3_and_x4_squared R²: 0.7957  MSE: 17.6338
Spec: x1_squared                     R²: 0.7952  MSE: 17.6760
Spec: x1_and_x2_squared              R²: 0.7947  MSE: 17.7233
Spec: x1_and_x2_and_x3_squared       R²: 0.7947  MSE: 17.7233
Spec: x5_squared                     R²: 0.7938  MSE: 17.7986
Spec: x1_squared_interaction         R²: 0.7932  MSE: 17.8472
Spec: polynomial_degree_3            R²: 0.7685  MSE: 19.9872
Spec: polynomial_degree_2            R²: 0.7682  MSE: 20.0056
Spec: poly_deg2_ignore_x3            R²: 0.7622  MSE: 20.5245
Spec: x1_and_x5_squares_interaction  R²: 0.6962  MSE: 26.22

### The best model is 'all_features_squares_interactions' with a R² of 97%. Let's fit this model and analyze its coefficients

In [54]:
# 1) Transform the training data for the "all_features_squares_interactions" spec
X_train_t = transform_data(X_train, 'all_features_squares_interactions')
X_test_t  = transform_data(X_test,  'all_features_squares_interactions')

# 2) Fit a linear regression model on that specification
model = LinearRegression()
model.fit(X_train_t, y_train)

# 3) Create a DataFrame listing each feature and its coefficient
coeffs_df = pd.DataFrame({
    'Feature': X_train_t.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

# Print the coefficients table
print(coeffs_df.to_string(index=False))

# 4) Print the model intercept
print("\nIntercept:", model.intercept_)


                Feature  Coefficient
             x1_squared     8.111226
             x2_squared     8.044411
        x1_x_x2_squared     4.133054
                     x1     3.317359
                x2_x_x3     2.245588
        x2_x_x3_squared     2.245588
             x3_squared     1.795218
        x3_x_x3_squared     1.795218
                     x3     1.795218
        x4_x_x2_squared     1.298972
                x1_x_x5     1.077665
                x1_x_x4     0.713558
x1_squared_x_x2_squared     0.679454
x1_squared_x_x3_squared     0.507564
        x3_x_x1_squared     0.507564
        x5_x_x2_squared     0.312359
                     x5     0.277009
        x2_x_x4_squared     0.236092
                     x4     0.171768
                x4_x_x5     0.057962
             x5_squared     0.034288
        x1_x_x4_squared     0.032483
x1_squared_x_x5_squared     0.006601
        x4_x_x4_squared     0.005040
        x5_x_x4_squared     0.002527
x3_squared_x_x4_squared     0.002464
 

**Model Description**  
The final model—labeled **`all_features_squares_interactions`**—is a linear regression model that uses:

- **All Original Features** $x_1$,$x_2$,$x_3$,$x_4$,$x_5$ 
- **Squared Terms** $x_i^2$ for each feature $x_i$  
- **All Pairwise Interactions** $x_i$ x $x_j$ for every pair of features $i, j$

By including both squared terms and pairwise products, this model can capture a variety of **nonlinear relationships** that a standard linear model could miss. Essentially, it’s a polynomial expansion of degree 2 across all five features.

---

**Notable Features**  
- Because we generate many new columns (one for each square and each interaction), we end up with a **large set of features**.  
- In practice, certain squared terms and interactions often dominate—for instance, if $x_1$ or $x_5$ is particularly influential, you might see large positive or negative coefficients for $x_1^2$, $x_5^2$, or $x_1 \times x_5$.  
- You can inspect the coefficients table to see which derived features have **highest magnitude** coefficients (positively or negatively).

---

**Model Performance and Assessment**  
- The best R² on the test set is **0.971**, meaning the model explains about **97% of the variance** in the outcome on held-out data.  
- The **Mean Squared Error (MSE)** is **2.488**, indicating (on average) how far off predictions are from true values (squared difference). A smaller MSE generally indicates better performance.  
- These metrics (R² and MSE) were obtained on a **test set** that was separated from the training data, providing a straightforward estimate of how well the model generalizes.  
- An R² of 0.97 suggests **very good fit** for the given dataset. 